# Searching for the Coffee Shops Location

The Battle of the Neighborhoods - Applied Data Science Capstone Project (Week 2)

## Table of contents
* [Introduction](#introduction)
* [Data Requirement & Acquisition](#data)
* [Analysis](#analysis)

## Introduction <a name="introduction"></a>

This project is purposed to find a good location for a Coffee Shop in **Bandung** Indonesia to meet the need of stakeholders whom have interest. In this project, the term **good location for a Coffee Shop** is interpreted as **areas where people are gathered** but **with no coffee shops and cafe around**. The area where people are gathered in this project is represented by the area that have a lot of venues. Some data science techniques and tools will be employed to generate the most suitable neighborhoods that meet the criteria. The result can be used as the consideration for the stakeholder in deciding the final location. 

## Data Requirement & Acquisition <a name="data"></a>

### Data Requirement
Based on the problem definition, factors that will influence our decission are:
* number of existing venue in the neighborhood (it indicates the crowds)
* inexistence of cofee shops and cafe in the neighborhood
* similarity of venue in the neighborhood

For these purpose, this project need the data as follows:
* Bandung with district and neighborhood’s data. This data should consist of all district in Bandung city. Each district should consist with all neighborhood with postcode and geocode data. This data is used as a reference data for constructing a map and as the initial location to be explore.
* Bandung venue data.

### Data Acquisition

In [35]:
#importing the required libraries

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


#### Bandung's Neighborhood Data

In [36]:
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

In [37]:
# The code was removed by Watson Studio for sharing.

In [38]:
#getting Bandung's Neighborhood data from .csv file
PostCode = pd.read_csv(body)
pd.options.display.float_format = '{:,.4f}'.format
print(PostCode.shape)
PostCode.head()

(151, 5)


,Post_Code,District,Neighborhood,Latitude,Longitude
0,40111,Sumur Bandung,Braga,-6.9176,107.6094
1,40112,Sumur Bandung,Kebon Pisang,-6.9189,107.6171
2,40113,Sumur Bandung,Merdeka,-6.9137,107.6201
3,40114,Bandung Wetan,Cihapit,-6.9083,107.6260
4,40115,Bandung Wetan,Citarum,-6.9035,107.6171


In [39]:
address = 'Alun-alun Bandung, Indonesia'

geolocator = Nominatim (user_agent="bandung_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bandung city center are  {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bandung city center are  -6.9211979, 107.6074903.


In [40]:
# create map of Bandung using latitude and longitude values
map_bandung = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(PostCode['Latitude'], PostCode['Longitude'], PostCode['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='green',
        fill=False,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bandung)  
    
map_bandung

#### Bandung's Venues Data
(Working with Foursquare API)

In [41]:
# The code was removed by Watson Studio for sharing.

In [42]:
# get neighborhood coordinates data

neighborhood_latitude = PostCode.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = PostCode.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = PostCode.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Braga are -6.917563900000001, 107.6093662.


In [48]:
radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
#url

In [51]:
results = requests.get(url).json()
#results

In [45]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [46]:
# clean json and create dataframe 
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
nearby_venues.head()

94 venues were returned by Foursquare.


,name,categories,lat,lng
0,éL Royale Hotel Bandung,Hotel,-6.9161,107.6106
1,Starbucks,Coffee Shop,-6.9170,107.6091
2,Braga Permai - Maison Bogerijen,Eastern European Restaurant,-6.9174,107.6094
3,Braga Punya Cerita,Café,-6.9172,107.6093
4,Hangover,Bar,-6.9186,107.6097


In [47]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

bandung_venues = getNearbyVenues(names=PostCode['Neighborhood'],
                                   latitudes=PostCode['Latitude'],
                                   longitudes=PostCode['Longitude']
                                  )

Braga
Kebon Pisang
Merdeka
Cihapit
Citarum
Tamansari
Babakan Ciamis
Cicadas
Sukamaju
Cihaur Geulis
Sukaluyu
Neglasari
Cikutra
Padasuka
Sukapada
Cipaganti
Lebak Gede
Lebak Siliwangi
Sadang Serang
Sekeloa
Dago
Hegarmanah
Ciumbuleuit
Ledeng
Sarijadi
Sukarasa
Geger Kalong
Isola
Pasteur
Cipedes
Sukabungah
Sukagalih
Sukawarna
Pasir Kaliki
Arjuna
Pajajaran
Pamoyanan
Husen Sastranegara
Sukaraja
Kebon Jeruk
Ciroyom
Dungus Cariang
Campaka
Garuda
Maleber (Maleer)
Cigadung
Pasirlayung
Jatihandap
Karang Pamulang
Pasir Impun
Sindang Jaya
Warung Muncang
Caringin
Cibuntu
Cijerah
Cigondewah Kaler
Cigondewah Kidul
Cigondewah Rahayu
Gempolsari
Sukahaji
Babakan
Babakan Ciparay
Margahayu Utara
Margasuka
Cirangrang
Jamika
Suka Asih
Babakan Asih
Babakan Tarogong
Kopo
Situsaeur
Kebon Lega
Cibaduyut
Mekarwangi
Cibaduyut Wetan
Cibaduyut Kidul
Cibadak
Karanganyar
Nyengseret
Panjunan
Karasak
Pelindung Hewan
Balong Gede
Ciateul
Pungkur
Cigereleng
Ancol
Pasirluyu
Ciseureuh
Wates
Cikawao
Paledang
Burangrang
Malabar


KeyError: 'groups'

In [52]:
# create new dataframe of venues in Bandung
print(bandung_venues.shape)
bandung_venues.head()

(2068, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Braga,-6.9176,107.6094,éL Royale Hotel Bandung,-6.9161,107.6106,Hotel
1,Braga,-6.9176,107.6094,Starbucks,-6.9170,107.6091,Coffee Shop
2,Braga,-6.9176,107.6094,Braga Permai - Maison Bogerijen,-6.9174,107.6094,Eastern European Restaurant
3,Braga,-6.9176,107.6094,Braga Punya Cerita,-6.9172,107.6093,Café
4,Braga,-6.9176,107.6094,Hangover,-6.9186,107.6097,Bar


In [53]:
bdgvenuenum = bandung_venues[['Venue Category', 'Venue']].groupby('Venue Category').count().reset_index()
print(bdgvenuenum.shape)
bdgvenuenum.sort_values ('Venue',ascending=False).head()

(217, 2)


,Venue Category,Venue
40,Café,122
114,Indonesian Restaurant,117
109,Hotel,108
48,Coffee Shop,98
44,Chinese Restaurant,80


In [54]:
# number of venues in each neighborhood
bdgvenuegrouped = bandung_venues[['Neighborhood', 'Venue']].groupby('Neighborhood').count().reset_index()
bdgvenuegrouped.sort_values ('Venue',ascending=False).head()

,Neighborhood,Venue
16,Braga,94
139,Tamansari,76
75,Kebon Jeruk,74
7,Babakan Ciamis,68
87,Malabar,62


In [55]:
# number of unique categories
print('There are {} uniques categories.'.format(len(bandung_venues ['Venue Category'].unique())))

There are 217 uniques categories.


In [56]:
bandung_onehot = pd.get_dummies(bandung_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bandung_onehot['Neighborhood'] = bandung_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bandung_onehot.columns[-1]] + list(bandung_onehot.columns[:-1])
bandung_onehot = bandung_onehot[fixed_columns]

print(bandung_onehot.shape)
bandung_onehot.head()

(2068, 218)


,Neighborhood,ATM,Accessories Store,Acehnese Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bathing Area,Beach Bar,Bed & Breakfast,Beer Garden,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Buffet,Building,Burger Joint,Business Service,Cafeteria,Café,Camera Store,Car Wash,Casino,Chinese Restaurant,Church,City Hall,Clothing Store,Coffee Shop,College Academic Building,College Auditorium,College Basketball Court,College Cafeteria,College Stadium,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doctor's Office,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indonesian Meatball Place,Indonesian Restaurant,Insurance Office,Internet Cafe,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Venue,Nightclub,Noodle House,Office,Optical Shop,Other Repair Shop,Outdoor Supply Store,Padangnese Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Salad Place,Salon / Barbershop,Satay Restaurant,Scenic Lookout,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stadium,Steakhouse,Sundanese Restaurant,Supermarket,Sushi Restaurant,Tailor Shop,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track,Track Stadium,Train,Train Station,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Wings Joint,Women's Store,Yoga Studio
0,Braga,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Braga,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Braga,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [57]:
# frequency of occurence of each category in each neighbourhood
bandung_grouped = bandung_onehot.groupby('Neighborhood').mean().reset_index()
print (bandung_grouped.shape)
bandung_grouped.head()

(143, 218)


,Neighborhood,ATM,Accessories Store,Acehnese Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bathing Area,Beach Bar,Bed & Breakfast,Beer Garden,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Buffet,Building,Burger Joint,Business Service,Cafeteria,Café,Camera Store,Car Wash,Casino,Chinese Restaurant,Church,City Hall,Clothing Store,Coffee Shop,College Academic Building,College Auditorium,College Basketball Court,College Cafeteria,College Stadium,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doctor's Office,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indonesian Meatball Place,Indonesian Restaurant,Insurance Office,Internet Cafe,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Venue,Nightclub,Noodle House,Office,Optical Shop,Other Repair Shop,Outdoor Supply Store,Padangnese Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Salad Place,Salon / Barbershop,Satay Restaurant,Scenic Lookout,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stadium,Steakhouse,Sundanese Restaurant,Supermarket,Sushi Restaurant,Tailor Shop,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track,Track Stadium,Train,Train Station,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Wings Joint,Women's Store,Yoga Studio
0,Ancol,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0

In [58]:
#merge to get number of venue
bandung_grouped_ven = bdgvenuegrouped
bandung_grouped_ven = bandung_grouped_ven.join(bandung_grouped.set_index('Neighborhood'), on='Neighborhood').dropna()

# Bandung's Venue Data
print (bandung_grouped_ven.shape)
bandung_grouped_ven.sort_values('Venue', ascending=False).head()

(143, 219)


,Neighborhood,Venue,ATM,Accessories Store,Acehnese Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bathing Area,Beach Bar,Bed & Breakfast,Beer Garden,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Buffet,Building,Burger Joint,Business Service,Cafeteria,Café,Camera Store,Car Wash,Casino,Chinese Restaurant,Church,City Hall,Clothing Store,Coffee Shop,College Academic Building,College Auditorium,College Basketball Court,College Cafeteria,College Stadium,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doctor's Office,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indonesian Meatball Place,Indonesian Restaurant,Insurance Office,Internet Cafe,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Venue,Nightclub,Noodle House,Office,Optical Shop,Other Repair Shop,Outdoor Supply Store,Padangnese Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Salad Place,Salon / Barbershop,Satay Restaurant,Scenic Lookout,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stadium,Steakhouse,Sundanese Restaurant,Supermarket,Sushi Restaurant,Tailor Shop,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track,Track Stadium,Train,Train Station,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Wings Joint,Women's Store,Yoga Studio
16,Braga,94,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0106,0.0000,0.0000,0.0106,0.0000,0.0000,0.0000,0.0319,0.0106,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0213,0.0106,0.0106,0.0000,0.0106,0.0000,0.0000,0.0106,0.0000,0.0000,0.0000,0.0000,0.0000,0.0957,0.0106,0.0000,0.0000,0.0213,0.0000,0.0000,0.0106,0.0638,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0213,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0213,0.0213,0.0106,0.0000,0.0000,0.0213,0.0319,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0106,0.0000,0.0000,0.0000,0.0000,0.0106,0.0106,0.0000,0.0000,0.0000,0.0851,0.0000,0.0000,0.0000,0.0106,0.0426,0.0000,0.0000,0.0000,0.0000,0.0000,0.0106,0.0000,0.0319,0.0000,0.0106,0.0213,0.0000,0.0000,0.0000,0.0000,0.0000,

## Analysis <a name="analysis"></a>

In [59]:
#delete area that have limited number of venues & area with too many cofee shops and cafe
feasible_area = bandung_grouped_ven.loc[(bandung_grouped_ven['Coffee Shop']== 0.00) & (bandung_grouped_ven['Venue']>= 5.00) & (bandung_grouped_ven['Café']== 0.00)]
#del feasible_area['Venue']
print (feasible_area.shape)
feasible_area.head()

(18, 219)


,Neighborhood,Venue,ATM,Accessories Store,Acehnese Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bathing Area,Beach Bar,Bed & Breakfast,Beer Garden,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Buffet,Building,Burger Joint,Business Service,Cafeteria,Café,Camera Store,Car Wash,Casino,Chinese Restaurant,Church,City Hall,Clothing Store,Coffee Shop,College Academic Building,College Auditorium,College Basketball Court,College Cafeteria,College Stadium,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doctor's Office,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indonesian Meatball Place,Indonesian Restaurant,Insurance Office,Internet Cafe,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Venue,Nightclub,Noodle House,Office,Optical Shop,Other Repair Shop,Outdoor Supply Store,Padangnese Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Salad Place,Salon / Barbershop,Satay Restaurant,Scenic Lookout,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stadium,Steakhouse,Sundanese Restaurant,Supermarket,Sushi Restaurant,Tailor Shop,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track,Track Stadium,Train,Train Station,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Wings Joint,Women's Store,Yoga Studio
3,Antapani Tengah,14,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0714,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0714,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0714,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0714,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0714,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0714,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000

In [60]:
feasible_area_novenue = feasible_area.drop('Venue',1)
feasible_area_novenue.head()

,Neighborhood,ATM,Accessories Store,Acehnese Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bathing Area,Beach Bar,Bed & Breakfast,Beer Garden,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Buffet,Building,Burger Joint,Business Service,Cafeteria,Café,Camera Store,Car Wash,Casino,Chinese Restaurant,Church,City Hall,Clothing Store,Coffee Shop,College Academic Building,College Auditorium,College Basketball Court,College Cafeteria,College Stadium,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doctor's Office,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indonesian Meatball Place,Indonesian Restaurant,Insurance Office,Internet Cafe,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Venue,Nightclub,Noodle House,Office,Optical Shop,Other Repair Shop,Outdoor Supply Store,Padangnese Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Salad Place,Salon / Barbershop,Satay Restaurant,Scenic Lookout,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stadium,Steakhouse,Sundanese Restaurant,Supermarket,Sushi Restaurant,Tailor Shop,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track,Track Stadium,Train,Train Station,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Wings Joint,Women's Store,Yoga Studio
3,Antapani Tengah,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0714,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0714,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0714,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0714,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0714,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0714,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,

In [65]:
num_top_venues = 5

for hood in feasible_area_novenue['Neighborhood']:
    #print("----"+hood+"----")
    temp = feasible_area_novenue[feasible_area_novenue['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

          venue   freq
0  Noodle House 0.1400
1      Mountain 0.0700
2        Bakery 0.0700
3      Pharmacy 0.0700
4    Restaurant 0.0700


                   venue   freq
0  Indonesian Restaurant 0.2200
1           Noodle House 0.0900
2       Asian Restaurant 0.0900
3            Tailor Shop 0.0400
4              Juice Bar 0.0400


           venue   freq
0  Train Station 0.3300
1     Hotel Pool 0.1700
2     Restaurant 0.1700
3         Arcade 0.1700
4  Grocery Store 0.1700


                  venue   freq
0  Sundanese Restaurant 0.2000
1  Gym / Fitness Center 0.2000
2      Basketball Court 0.2000
3    Chinese Restaurant 0.2000
4                  Pool 0.2000


              venue   freq
0        Donut Shop 0.2000
1  Department Store 0.2000
2     Historic Site 0.2000
3            Arcade 0.2000
4            Bakery 0.2000


               venue   freq
0         Food Truck 0.1400
1  Convenience Store 0.1400
2   Insurance Office 0.1400
3        Supermarket 0.1400
4           Pharmacy 0.1400


In [62]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [66]:
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
feasible_area_sorted = pd.DataFrame(columns=columns)
feasible_area_sorted['Neighborhood'] = feasible_area_novenue['Neighborhood']

for ind in np.arange(feasible_area_novenue.shape[0]):
    feasible_area_sorted.iloc[ind, 1:] = return_most_common_venues(feasible_area.iloc[ind, :], num_top_venues)
print(feasible_area_sorted.shape)
feasible_area_sorted.head()

(18, 6)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Antapani Tengah,Noodle House,Pharmacy,Restaurant,Breakfast Spot,Supermarket
4,Arjuna,Indonesian Restaurant,Noodle House,Asian Restaurant,Flower Shop,Seafood Restaurant
9,Babakan Penghulu,Train Station,Restaurant,Hotel Pool,Grocery Store,Arcade
26,Cibuntu,Pool,Sundanese Restaurant,Basketball Court,Gym / Fitness Center,Chinese Restaurant
30,Cigending,Historic Site,Arcade,Department Store,Bakery,Donut Shop


In [67]:
# clustering
kclusters = 4

feasible_area_clustering = feasible_area.drop('Neighborhood',1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(feasible_area_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 3, 3, 3, 0, 3, 3, 3, 2], dtype=int32)

In [68]:
# add clustering labels
#del feasible_area_sorted['Cluster Labels']
feasible_area_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
feasible_area_sorted.head()
feasible_area_location = PostCode

# merge to get geocode
feasible_area_location = feasible_area_location.join(feasible_area_sorted.set_index('Neighborhood'), on='Neighborhood').dropna()
feasible_area_location['Cluster Labels'] = feasible_area_location['Cluster Labels'].astype('int32')

feasible_area_location = feasible_area_location.join(bdgvenuegrouped.set_index('Neighborhood'), on='Neighborhood').dropna()

print (feasible_area_location.shape)
feasible_area_location.sort_values('Cluster Labels',ascending=True)

(19, 12)


,Post_Code,District,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Venue
126,40293,Arcamanik,Sindang Jaya,-6.9035,107.6852,0,Seafood Restaurant,Arcade,Food & Drink Shop,Concert Hall,Dog Run,8
30,40162,Sukajadi,Sukabungah,-6.8941,107.5934,0,Indonesian Restaurant,Diner,Hotel,Food Truck,Convenience Store,8
35,40173,Cicendo,Pajajaran,-6.8995,107.5875,0,Indonesian Restaurant,Department Store,High School,Hotel,Sundanese Restaurant,9
50,40195,Mandalajati,Sindang Jaya,-6.9035,107.6852,0,Seafood Restaurant,Arcade,Food & Drink Shop,Concert Hall,Dog Run,8
85,40253,Regol,Cigereleng,-6.9398,107.6112,0,Food Truck,Burger Joint,Pharmacy,Convenience Store,Supermarket,7
71,40235,Bojongloa Kidul,Kebon Lega,-6.9479,107.5993,0,Indonesian Restaurant,Stadium,Car Wash,Road,Monument / Landmark,7
70,40234,Bojongloa Kidul,Situsaeur,-6.9403,107.5964,0,Hotel,Farmers Market,Gym / Fitness Center,Indonesian Restaurant,Snack Place,7
34,40172,Cicendo,Arjuna,-6.9095,107.5934,1,Indonesian Restaurant,Noodle House,Asian Restaurant,Flower Shop,Seafood Restaurant,23
18,40133,Coblong,Sadang Serang,-6.8955,107.6230,2,Indonesian Restaurant,Playground,Park,Food Truck,Noodle House,11
120,40291,Antapani,Antapani Tengah,-6.9166,107.6607,2,Noodle House,Pharmacy,Restaurant,Breakfast Spot,Supermarket,14


In [50]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(feasible_area_location['Latitude'], feasible_area_location['Longitude'], feasible_area_location['Neighborhood'], feasible_area_location['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(map_clusters)
       
map_clusters

#### End 
created by: Fahmi